In [0]:
import keras
import numpy as np
from keras.applications import resnet50
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.imagenet_utils import decode_predictions
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

resnet_model = resnet50.ResNet50(weights='imagenet')

Using TensorFlow backend.


102858752/102853048 [==============================] - 1s 0us/step


In [0]:
import glob
import numpy as np
import pandas as pd
import os
import shutil 
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img

In [4]:
IMG_WIDTH=300
IMG_HEIGHT=300
IMG_DIM = (IMG_WIDTH, IMG_HEIGHT)

validation_files = glob.glob('/content/drive/My Drive/pets/test/*')
validation_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in validation_files]
validation_imgs = np.array(validation_imgs)
validation_labels = [fn.split('/')[-1].split('.')[0].strip() for fn in validation_files]

train_files = glob.glob('/content/drive/My Drive/pets/train/*') 
train_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in train_files]
train_imgs = np.array(train_imgs)
train_labels = [fn.split('/')[-1].split('.')[0].strip() for fn in train_files]


print('Train dataset shape:', train_imgs.shape, 
 '\tValidation dataset shape:', validation_imgs.shape)

Train dataset shape: (5900, 300, 300, 3) 	Validation dataset shape: (1463, 300, 300, 3)


In [0]:
train_imgs_scaled = train_imgs.astype('float32') 
validation_imgs_scaled = validation_imgs.astype('float32') 
train_imgs_scaled /= 255 
validation_imgs_scaled /= 255 

In [0]:
# encode text category labels 
from sklearn.preprocessing import LabelEncoder 
 
le = LabelEncoder() 
le.fit(train_labels) 
train_labels_enc = le.transform(train_labels) 
validation_labels_enc = le.transform(validation_labels) 
 


NUM_CLASSES = 37
train_labels = keras.utils.to_categorical(train_labels_enc, NUM_CLASSES)
validation_labels = keras.utils.to_categorical(validation_labels_enc, NUM_CLASSES)
#print(train_labels[60:70], train_labels_enc[60:70])

In [0]:
train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, rotation_range=50,
 width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, 
 horizontal_flip=True, fill_mode='nearest')
val_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
train_generator = train_datagen.flow(train_imgs, train_labels,batch_size=30)
val_generator = val_datagen.flow(validation_imgs, validation_labels, batch_size=30)

In [27]:
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras
restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(IMG_HEIGHT,IMG_WIDTH,3))
output = restnet.layers[-1].output
output = keras.layers.Flatten()(output)
restnet = Model(restnet.input, output=output)
for layer in restnet.layers:
    layer.trainable = False
#restnet.summary()

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(Tensor("in..., outputs=Tensor("fl...)`
  import sys


In [40]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers
model = Sequential()
model.add(restnet)
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(37, activation='softmax'))

opt=optimizers.RMSprop(lr=2e-5)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_3 (Model)              (None, 204800)            23587712  
_________________________________________________________________
dense_22 (Dense)             (None, 512)               104858112 
_________________________________________________________________
dropout_15 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_16 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 37)                18981     
Total params: 128,727,461
Trainable params: 105,139,749
Non-trainable params: 23,587,712
_______________________________

In [0]:
#history = model.fit_generator(train_generator, 
#                              steps_per_epoch=20, 
#                              epochs=20,
#                              validation_data=val_generator, 
#                              validation_steps=10, 
#                              verbose=1)


history = model.fit(train_imgs_scaled, train_labels,
                    epochs=10, 
                    batch_size=128,
                    validation_data=(validation_imgs_scaled, validation_labels),
                    shuffle=True,verbose=1)

Train on 5900 samples, validate on 1463 samples
Epoch 1/10
5900/5900 [==============================] - 1360s 231ms/step - loss: 8.1999 - accuracy: 0.1271 - val_loss: 6.1835 - val_accuracy: 0.0253
Epoch 2/10
5900/5900 [==============================] - 1384s 235ms/step - loss: 3.3750 - accuracy: 0.2661 - val_loss: 4.6551 - val_accuracy: 0.0253
Epoch 3/10
5900/5900 [==============================] - 1392s 236ms/step - loss: 3.2100 - accuracy: 0.3324 - val_loss: 3.6474 - val_accuracy: 0.0232
Epoch 4/10
5900/5900 [==============================] - 1385s 235ms/step - loss: 3.1197 - accuracy: 0.3708 - val_loss: 3.6131 - val_accuracy: 0.0273
Epoch 5/10
5900/5900 [==============================] - 1385s 235ms/step - loss: 2.8046 - accuracy: 0.3975 - val_loss: 3.6151 - val_accuracy: 0.0273
Epoch 6/10
 640/5900 [==>...........................] - ETA: 16:22 - loss: 2.6859 - accuracy: 0.4469

In [34]:
print(train_imgs_scaled.shape)
print(train_labels.shape)
print(validation_imgs_scaled.shape)
print(validation_labels.shape)

(5900, 300, 300, 3)
(5900, 37)
(1463, 300, 300, 3)
(1463, 37)
